# Test Model
- test model on holdout sessions

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix

In [ ]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
n_hl = 10

# Define Model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(300, n_hl),
            nn.ReLU(),
            nn.Linear(n_hl, 1)
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits  

model = MLP().to(device)
model.load_state_dict(torch.load('model/model.pt'))
    
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Test by Predicting with Model

# Get one file
index = 36
step = 10000    # for memory

# Read in holdout session
dataset = torch.load(f'pipeline/holdouts/{index}-holdout.pt')
session_length = len(dataset)
dataloader = DataLoader(dataset, batch_size=step)
y_true = dataset[:][1]

model.eval()

preds = []
n_correct = 0
loss = 0

for X, y in dataloader:
    X = X.to(device) 
    y = y.to(device)

    logits = model(X)
    pred = torch.round(nn.Sigmoid()(logits))

    n_correct += sum(y == pred)
    preds += pred.flatten().tolist()
    loss += criterion(logits, y).item()

y_pred = np.array(preds).reshape(-1,1)
accuracy = (n_correct / session_length).item()
loss = loss/len(dataloader)
print(f'Accuracy: {100*accuracy:.4}%')
print(f'Loss: {loss:.4}')

In [ ]:
# Visualize predictions
df = pd.read_csv(f'pipeline/1_dm/{index}.csv')
df['y_true'] = np.pad(y_true.flatten()*10, (0, 99), mode='constant', constant_values=0)
df['y_pred'] = np.pad(y_pred.flatten()*10, (0, 99), mode='constant', constant_values=0)

figure = px.line(df)
figure.show(renderer='browser')

In [ ]:
# Heatmap

y_true = dataset[:][1]

fig,axes = plt.subplots(1,3,sharey=True,figsize=(10,5))
sns.heatmap(confusion_matrix(y_true=y_true,y_pred=y_pred,normalize='true'),annot=True,ax=axes[0],cbar=False,fmt='.2f')
sns.heatmap(confusion_matrix(y_true=y_true,y_pred=y_pred,normalize='pred'),annot=True,ax=axes[1],cbar=False,fmt='.2f')
sns.heatmap(confusion_matrix(y_true=y_true,y_pred=y_pred),annot=True,ax=axes[2],cbar=False,fmt='.2f')
axes[0].set_title('Recall')
axes[1].set_title('Precision')
axes[2].set_title('Count')
plt.savefig(f'cm.jpg',dpi=200,bbox_inches='tight')